In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [4]:
website = 'https://www.citizensinformation.ie/en/health/health-system/'
website = 'https://www.citizensinformation.ie/en/health/health-system/private-health-insurance/'
website = 'https://www.citizensinformation.ie/en/'

headers = {
    'User-Agent': 'My Scraper Bot (contact: [allende.rev@gmail.com](mailto:allende.rev@gmail.com))'
}

page_to_scrape = requests.get(website, headers=headers)
soup = BeautifulSoup(page_to_scrape.content, 'html.parser')

In [35]:
main_section = soup.find('div', {'class': 'page_content'})
links = main_section.select('a[href^="/en/"]')

In [38]:
links

[<a href="/en/health/">Health</a>,
 <a href="/en/health/covid19/">COVID-19, </a>,
 <a href="/en/health/health-system/">Health system, </a>,
 <a href="/en/health/medical-cards-and-gp-visit-cards/">Medical cards and GP visit cards, </a>,
 <a href="/en/health/health-services/">Health services, </a>,
 <a href="/en/health/health-services/gp-and-hospital-services/">GP and hospital services, </a>,
 <a href="/en/health/health-services/dental-aural-and-optical-services/">Dental, aural and optical health, </a>,
 <a href="/en/health/health-services/children-s-health/">Children's health, </a>,
 <a href="/en/health/health-services/health-services-for-older-people/">Health services for older people</a>,
 <a href="/en/social-welfare/">Social Welfare</a>,
 <a href="/en/social-welfare/irish-social-welfare-system/">Irish social welfare system, </a>,
 <a href="/en/social-welfare/irish-social-welfare-system/claiming-a-social-welfare-payment/">Claiming a social welfare payment, </a>,
 <a href="/en/social-w

In [40]:
[link for link in soup.select('a[href^="/en/"]') if '/' not in link['href'][5:]]

[<a href="/en/" title="Home">
 <img alt="Citizens Information logo" class="logo" src="/static/images/cio_logo.jpg"/>
 </a>]

In [34]:
main_section

In [30]:
soup.select('a[href^="/en/"]')

[<a href="/en/" title="Home">
 <img alt="Citizens Information logo" class="logo" src="/static/images/cio_logo.jpg"/>
 </a>,
 <a href="/en/health/">Health</a>,
 <a href="/en/social-welfare/">Social Welfare</a>,
 <a href="/en/employment/">Employment</a>,
 <a href="/en/money-and-tax/">Money and Tax</a>,
 <a href="/en/moving-country/">Moving Country</a>,
 <a href="/en/returning-to-ireland/">Returning to Ireland</a>,
 <a href="/en/housing/">Housing</a>,
 <a href="/en/education/">Education and Training</a>,
 <a href="/en/travel-and-recreation/">Travel and Recreation</a>,
 <a href="/en/environment/">Environment</a>,
 <a href="/en/consumer/">Consumer</a>,
 <a href="/en/government-in-ireland/">Government in Ireland </a>,
 <a href="/en/death/">Death and Bereavement</a>,
 <a href="/en/birth-family-relationships/">Family and Relationships</a>,
 <a href="/en/justice/">Justice</a>,
 <a href="/en/social-welfare/disability-and-illness/checklist-disability/">I am a person with a disability</a>,
 <a hre

In [17]:
soup


<!DOCTYPE html>

<html lang="en">
<head>
<title>Home</title>
<meta content="Citizensinformation.ie" name="author"/>
<meta content="Citizensinformation.ie" name="dc.publisher"/>
<meta content="Citizensinformation.ie" name="dc.creator"/>
<meta content="Ireland" name="dc.coverage"/>
<meta content="About Citizensinformation.ie" name="dc.subject"/>
<meta content="lYaVni0QODbsrJ5-aFH_mmHfBAKhDqfqgIbFKSmMACs" name="google-site-verification">
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<link href="/static/styles/print.css" media="print" rel="stylesheet" type="text/css"/>
<link href="/static/styles/screen.css?20240331" media="screen" rel="stylesheet" type="text/css"/>
<link href="/static/styles/screen_added.css?20230619" media="screen" rel="stylesheet" type="text/css"/>
<link href="/static/styles/portal-custom.css?20211209" media="screen" rel="stylesheet" type="text/css"/>
<!--[if IE 9]>
    <link rel="stylesheet" type="text/css" href="/static/styles/ie9.css">
    <scr

In [14]:
categories = soup.find_all('a', {'class': 'top-level-category'})

In [12]:
soup.find_all({'href="/en/housing/renting'})

[]

In [7]:
sections = soup.find_all('section', class_='category')

# Traverse the tree-like structure
for section in sections:
    category_name = section.find('h2').text.strip()
    print(f"Category: {category_name}")
    
    # Get the subtopics (links) within the category
    subtopics = section.find_all('a')
    for subtopic in subtopics:
        subtopic_name = subtopic.text.strip()
        subtopic_url = subtopic['href']
        print(f"  - {subtopic_name} ({subtopic_url})")

Category: Health
  - Health (/en/health/)
  - COVID-19, (/en/health/covid19/)
  - Health system, (/en/health/health-system/)
  - Medical cards and GP visit cards, (/en/health/medical-cards-and-gp-visit-cards/)
  - Health services, (/en/health/health-services/)
  - GP and hospital services, (/en/health/health-services/gp-and-hospital-services/)
  - Dental, aural and optical health, (/en/health/health-services/dental-aural-and-optical-services/)
  - Children's health, (/en/health/health-services/children-s-health/)
  - Health services for older people (/en/health/health-services/health-services-for-older-people/)
Category: Social Welfare
  - Social Welfare (/en/social-welfare/)
  - Irish social welfare system, (/en/social-welfare/irish-social-welfare-system/)
  - Claiming a social welfare payment, (/en/social-welfare/irish-social-welfare-system/claiming-a-social-welfare-payment/)
  - Social insurance (PRSI), (/en/social-welfare/irish-social-welfare-system/social-insurance-prsi/)
  - Soci

In [8]:
sections

[<section aria-labelledby="categorylist" class="category" role="complementary">
 <div class="contents">
 <h2>
 <a href="/en/health/">Health</a>
 </h2>
 <p>
 <a href="/en/health/covid19/">COVID-19, </a>
 <a href="/en/health/health-system/">Health system, </a>
 <a href="/en/health/medical-cards-and-gp-visit-cards/">Medical cards and GP visit cards, </a>
 <a href="/en/health/health-services/">Health services, </a>
 <a href="/en/health/health-services/gp-and-hospital-services/">GP and hospital services, </a>
 <a href="/en/health/health-services/dental-aural-and-optical-services/">Dental, aural and optical health, </a>
 <a href="/en/health/health-services/children-s-health/">Children's health, </a>
 <a href="/en/health/health-services/health-services-for-older-people/">Health services for older people</a>
 </p>
 </div>
 </section>,
 <section aria-labelledby="categorylist" class="category" role="complementary">
 <div class="contents">
 <h2>
 <a href="/en/social-welfare/">Social Welfare</a>
 

In [6]:
soup.find_all('section', class_='category')

[<section aria-labelledby="categorylist" class="category" role="complementary">
 <div class="contents">
 <h2>
 <a href="/en/health/">Health</a>
 </h2>
 <p>
 <a href="/en/health/covid19/">COVID-19, </a>
 <a href="/en/health/health-system/">Health system, </a>
 <a href="/en/health/medical-cards-and-gp-visit-cards/">Medical cards and GP visit cards, </a>
 <a href="/en/health/health-services/">Health services, </a>
 <a href="/en/health/health-services/gp-and-hospital-services/">GP and hospital services, </a>
 <a href="/en/health/health-services/dental-aural-and-optical-services/">Dental, aural and optical health, </a>
 <a href="/en/health/health-services/children-s-health/">Children's health, </a>
 <a href="/en/health/health-services/health-services-for-older-people/">Health services for older people</a>
 </p>
 </div>
 </section>,
 <section aria-labelledby="categorylist" class="category" role="complementary">
 <div class="contents">
 <h2>
 <a href="/en/social-welfare/">Social Welfare</a>
 